# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921693383033                   -0.69    6.0         
  2   -7.926163680959       -2.35       -1.22    1.0    253ms
  3   -7.926835555579       -3.17       -2.37    1.6    280ms
  4   -7.926861518868       -4.59       -3.03    2.5    296ms
  5   -7.926861641039       -6.91       -3.40    1.8    262ms
  6   -7.926861669316       -7.55       -3.79    2.0    301ms
  7   -7.926861680385       -7.96       -4.33    1.2    237ms
  8   -7.926861681745       -8.87       -4.83    2.0    265ms
  9   -7.926861681858       -9.95       -5.24    1.8    262ms
 10   -7.926861681871      -10.90       -5.80    1.8    261ms
 11   -7.926861681873      -11.76       -6.59    2.2    308ms
 12   -7.926861681873      -13.21       -7.57    2.1    285ms
 13   -7.926861681873      -15.05       -7.89    3.0    304ms
 14   -7.926861681873      -15.05       -8.70    1.6    252ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921687150651                   -0.69    5.6         
  2   -7.926166765078       -2.35       -1.22    1.0    255ms
  3   -7.926837080908       -3.17       -2.37    1.9    298ms
  4   -7.926861490053       -4.61       -3.03    2.6    300ms
  5   -7.926861640302       -6.82       -3.39    2.1    270ms
  6   -7.926861669391       -7.54       -3.78    1.8    327ms
  7   -7.926861680323       -7.96       -4.31    1.6    253ms
  8   -7.926861681669       -8.87       -4.70    1.9    264ms
  9   -7.926861681856       -9.73       -5.24    1.8    266ms
 10   -7.926861681870      -10.83       -5.74    2.0    270ms
 11   -7.926861681873      -11.68       -6.83    1.6    300ms
 12   -7.926861681873      -13.15       -7.46    3.2    324ms
 13   -7.926861681873   +    -Inf       -7.98    2.0    278ms
 14   -7.926861681873   +  -14.75       -8.73    2.0    285ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921666125516                   -0.69    5.9         
  2   -7.926165221586       -2.35       -1.22    1.0    339ms
  3   -7.926840110199       -3.17       -2.37    1.6    294ms
  4   -7.926864895045       -4.61       -3.02    2.9    332ms
  5   -7.926865050081       -6.81       -3.37    1.9    280ms
  6   -7.926865078211       -7.55       -3.74    1.6    261ms
  7   -7.926865091540       -7.88       -4.33    1.2    247ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
